# 02. Traffic Sign Classification - 모델 평가

이 노트북은 학습된 모델의 성능을 평가하고 다양한 지표를 확인합니다.

## 평가 과정
1. 환경 설정 및 학습된 모델 로드
2. 테스트 데이터셋 준비
3. 모델 평가 수행
4. 성능 지표 분석 (Accuracy, Precision, Recall, F1-Score)
5. Confusion Matrix 시각화

## 1. 환경 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers pillow torch torchvision scikit-learn matplotlib seaborn -q

In [ ]:
import os
import torch
import numpy as np
from PIL import Image
from pathlib import Path
from torch.utils.data import Dataset as TorchDataset
from transformers import AutoImageProcessor, AutoModelForImageClassification
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, classification_report
)
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ 라이브러리 로드 완료")

## 2. Google Drive 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. 작업 디렉토리 및 모델 경로 설정

**중요**: `WORK_DIR`을 본인의 Google Drive 경로로 수정하세요.

In [ ]:
# Google Drive 내 작업 디렉토리 경로 설정
WORK_DIR = '/content/drive/MyDrive/2026_AI_Advanced_Study-main/3차시/02_Traffic_Sign_Classification'

# 학습된 모델 경로 (고정)
MODEL_PATH = os.path.join(WORK_DIR, 'runs/classification/final_model')

# 작업 디렉토리로 이동
os.chdir(WORK_DIR)
print(f"현재 작업 디렉토리: {os.getcwd()}")

# 모델 파일 존재 확인
if os.path.exists(MODEL_PATH):
    print(f"✅ 모델 발견: {MODEL_PATH}")
else:
    print(f"❌ 모델을 찾을 수 없습니다: {MODEL_PATH}")
    print("   먼저 01_train.ipynb를 실행하여 모델을 학습하세요.")

## 4. 커스텀 데이터셋 클래스 정의

In [ ]:
class TrafficSignDataset(TorchDataset):
    """로컬 이미지 파일을 로드하는 커스텀 데이터셋"""
    
    def __init__(self, data_dir, processor=None):
        self.data_dir = Path(data_dir)
        self.processor = processor
        self.samples = []
        
        # 클래스별 이미지 수집
        for class_idx in range(5):
            class_dir = self.data_dir / f'class_{class_idx}'
            if class_dir.exists():
                for img_path in sorted(class_dir.glob('*.jpg')):
                    self.samples.append((img_path, class_idx))
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert('RGB')
        return {'image': image, 'label': label}

print("✅ 커스텀 데이터셋 클래스 정의 완료")

## 5. 모델 및 프로세서 로드

In [ ]:
print("🤖 모델 로드 중...")
model = AutoModelForImageClassification.from_pretrained(MODEL_PATH)
processor = AutoImageProcessor.from_pretrained(MODEL_PATH)

model.eval()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(f"✅ 모델 로드 완료 (Device: {device})")
print(f"📋 클래스 정보: 5개 교통 표지판 클래스 (Class 0 ~ Class 4)")

## 6. 테스트 데이터 준비

In [ ]:
print("📥 테스트 데이터셋 로드 중...")

# 테스트 데이터셋 생성
test_dataset = TrafficSignDataset('data/images/test', processor=None)

print(f"✅ 테스트 데이터 준비 완료: {len(test_dataset)}장")

## 7. 모델 평가 수행

In [ ]:
print("🔍 평가 시작...\n")

all_predictions = []
all_labels = []

with torch.no_grad():
    for item in test_dataset:
        image = item['image']
        label = item['label']
        
        # 전처리
        inputs = processor(images=image, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # 예측
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=-1)
        
        all_predictions.append(predictions.cpu().item())
        all_labels.append(label)

all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

print("✅ 평가 완료!")

## 8. 성능 지표 계산

In [ ]:
# 전체 성능
accuracy = accuracy_score(all_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(
    all_labels, all_predictions, average='weighted', zero_division=0
)

print("\n" + "="*50)
print("=== 전체 성능 (Overall Performance) ===")
print("="*50)
print(f"\n📊 Accuracy: {accuracy:.4f}")
print(f"📊 Precision: {precision:.4f}")
print(f"📊 Recall: {recall:.4f}")
print(f"📊 F1-Score: {f1:.4f}")

# 클래스별 성능
print("\n" + "="*50)
print("=== 클래스별 성능 (Per-Class Performance) ===")
print("="*50)

precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
    all_labels, all_predictions, average=None, zero_division=0
)

for i in range(5):
    print(f"\n📌 Class {i}:")
    print(f"   - Precision: {precision_per_class[i]:.4f}")
    print(f"   - Recall: {recall_per_class[i]:.4f}")
    print(f"   - F1-Score: {f1_per_class[i]:.4f}")
    print(f"   - Support: {support[i]}개")

## 9. Confusion Matrix 시각화

In [ ]:
# Confusion Matrix 계산
cm = confusion_matrix(all_labels, all_predictions)

# 시각화
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=[f'Class {i}' for i in range(5)],
            yticklabels=[f'Class {i}' for i in range(5)])
plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()

# 저장
os.makedirs('runs/classification_val', exist_ok=True)
plt.savefig('runs/classification_val/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Confusion Matrix 저장: runs/classification_val/confusion_matrix.png")

## 10. 상세 분류 리포트

In [ ]:
# Classification Report
print("\n" + "="*50)
print("=== 상세 분류 리포트 ===")
print("="*50)

target_names = [f'Class {i}' for i in range(5)]
report = classification_report(all_labels, all_predictions, 
                               target_names=target_names, 
                               zero_division=0)
print(report)

## 11. 결과 저장

In [ ]:
# 결과를 텍스트 파일로 저장
from datetime import datetime

results_file = 'runs/classification_val/test_results.txt'

with open(results_file, 'w', encoding='utf-8') as f:
    f.write("Traffic Sign Classification - 평가 결과\n")
    f.write(f"생성 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"모델 경로: {MODEL_PATH}\n")
    f.write(f"테스트 샘플 수: {len(all_labels)}\n\n")
    
    f.write("=== Overall Performance ===\n")
    f.write(f"Accuracy: {accuracy:.4f}\n")
    f.write(f"Precision: {precision:.4f}\n")
    f.write(f"Recall: {recall:.4f}\n")
    f.write(f"F1-Score: {f1:.4f}\n\n")
    
    f.write("=== Per-Class Performance ===\n")
    for i in range(5):
        f.write(f"\nClass {i}:\n")
        f.write(f"  Precision: {precision_per_class[i]:.4f}\n")
        f.write(f"  Recall: {recall_per_class[i]:.4f}\n")
        f.write(f"  F1-Score: {f1_per_class[i]:.4f}\n")
        f.write(f"  Support: {support[i]}\n")
    
    f.write("\n=== Classification Report ===\n")
    f.write(report)

print(f"✅ 평가 결과 저장: {results_file}")
print(f"\n다음 단계:")
print("  - 03_visualize.ipynb에서 예측 결과 시각화")